# Mixed Effects Models

In this notebook, we'll use the `statsmodels` library to fit mixed effects models to our data. Be careful, there is one cell that is very long to run (you will find a caution before it).

In [1]:
# import libraries
import altair as alt
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

# import other functions
from scripts_misc.imputer import *
from scripts_misc.feature_eng import *
from scripts_misc.drop import *
from scripts_misc.preprocessing_old import *

In [2]:
def report_performance(model, X_train, y_train, X_valid, y_valid, mode='mean', floor=False):
    """
    Evaluate train and validation performance on a fitted model.
    
    Parameters
    ---------     
    model: sklearn.ensemble._gb.GradientBoostingRegressor
        scikit-learn model
    X_train: pandas.core.frame.DataFrame
        X of training set
    y_train: pandas.core.series.Series
        y of training set
    X_valid: pandas.core.frame.DataFrame        
        X of validation set
    y_valid: pandas.core.series.Series
        y of validation set     
    mode: string
        'mean' or 'median'
    floor : boolean
        if true, all the negative values are turned into 0s
    
    Returns
    -------
    errors: list
        
    """
    if mode == 'mean':
        if floor:
            errors = [np.sqrt(metrics.mean_squared_error(y_train, list(map(lambda x: 0 if x<0 else x, model.predict(X_train))))), 
                  np.sqrt(metrics.mean_squared_error(y_valid, list(map(lambda x: 0 if x<0 else x, model.predict(X_valid)))))]
            metric = 'root mean squared'
        else:
            errors = [np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))), 
                      np.sqrt(metrics.mean_squared_error(y_valid, model.predict(X_valid)))]
            metric = 'root mean squared'
        
    elif mode == 'median':
        if floor:
            errors = [metrics.mean_absolute_error(y_train, list(map(lambda x: 0 if x<0 else x, model.predict(X_train)))), 
                      metrics.mean_absolute_error(y_valid, list(map(lambda x: 0 if x<0 else x, model.predict(X_valid))))]
            metric= 'mean absolute'
        else:
            errors = [metrics.mean_absolute_error(y_train, model.predict(X_train)), 
                      metrics.mean_absolute_error(y_valid, model.predict(X_valid))]
            metric= 'mean absolute'
    
    print('Training', metric, 'error:', errors[0])
    print('Validation', metric, 'error:', errors[1])
        
    return(errors)

## Preprocessing

In [15]:
#Download the data
df = pd.read_csv('../data/train_data.zip')
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1804425,8,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,90.0,244.20,0.157475,0.009783,0.147692
1,1812706,2,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
2,1812706,3,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
3,1812706,11,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,24.0,258.95,0.157475,0.009783,0.147692
4,1812706,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,258.95,0.157475,0.009783,0.147692


In [7]:
def preprocessing(group=[]):
    """
    Preprocess the data but doesn't run OHE on the column used as an argument
    
    Parameters
    ---------     
    group : list of one string representing either 'income_class', 'density_class' or 'climate'
    
    Returns
    -------
    (X_train_valid, y_train, y_valid) : a tuple with a tuple containing the preprocessed X_train and X_valid, 
                                        the preprocessed y_train and the preprocessed y_valid
        
    """
    df = pd.read_csv('../data/train_data.zip')
    df = drop_missing_unacast(df)
    # create X and y
    X = df.drop('unacast_session_count', axis=1)
    y = df.loc[:, 'unacast_session_count']
    # split the data
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                        test_size=0.2,
                                                          random_state=2020)
    # impute NaN values
    result = impute_data(X_train, X_valid)
    X_train = result[0]
    X_valid = result[1] 
    # perform feature eng
    X_train = comb_cols(X_train)
    X_valid = comb_cols(X_valid)
    # perform dropping
    X_train = drop_columns(X_train)
    X_valid = drop_columns(X_valid)
    X_train = X_train.drop(columns=['monthly_rain', 'monthly_avg_rain_length'])
    X_valid = X_valid.drop(columns=['monthly_rain', 'monthly_avg_rain_length'])

    #perform scaling of the numerical variables
    categorical_features = X_train.loc[:, X_train.dtypes == "object"]
    categorical_features.columns
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.drop(columns = list(categorical_features.columns)))
    X_valid_scaled = scaler.transform(X_valid.drop(columns = list(categorical_features.columns)))

    X_train = pd.concat([pd.DataFrame(X_train_scaled, 
                                      index=X_train.index, 
                                      columns=X_train.drop(columns = list(categorical_features.columns)).columns), 
                         X_train[list(categorical_features.columns)]], axis=1)
    X_valid = pd.concat([pd.DataFrame(X_valid_scaled, 
                                      index=X_valid.index, 
                                      columns=X_train.drop(columns = list(categorical_features.columns)).columns), 
                         X_valid[list(categorical_features.columns)]], axis=1)


    # perform OHE (climate, density_class, income_class)
    list_cat = ['income_class', 'density_class', 'climate']
    for cat in list_cat:
        if cat in group:
            list_cat.remove(cat)
    X_train_valid = clean_categorical(X_train, X_valid, list_cat)
    return(X_train_valid, y_train, y_valid)

In [44]:
X_train_valid=preprocessing(['climate'])
X_train = X_train_valid[0]
X_valid = X_train_valid[1]


print(X_train.shape)
print(X_valid.shape)

(39592, 628)
(9898, 628)


In [45]:
#concatenate X and y for the train
train = pd.concat([X_train, y_train], axis=1)

**Caution** : The following cell takes a very long time to run (2 days)

In [68]:
#Find the columns that make the fit function fail
# i and to_drop were updated with the last values the function would gave everytime the connection with the kernel would stop and the algorithm would stop running
i= 142
to_drop=['unacast_session_count', 'monthly_repeated_sessions', 'B23008e21', 'B23008e23', 'B12001e11', 'B23008e26', 
         'B11005e14', 'B11005e17', 'B20004e9', 'B19101e7', 'B19101e4', 'B12001e15', 'B09002e19', 'B23008e27', 'B20004e7', 
         'B11016e10', 'B17012e31', 'B19101e8', 'C18108e5', 'B25012e8', 'B25012e15', 'B17020e4', 'B25012e16', 'B25012e10', 
         'B25012e6', 'B09002e17', 'B25012e12', 'B25012e11', 'B08301e6', 'B17012e6', 'B19101e6', 'B25012e4', 'B19101e12', 
         'B17020e5', 'B23008e18', 'B19101e5', 'B12001e16', 'B08303e12', 'B17020e11', 'B10010e1', 'B19101e11', 'B08303e13', 
         'B16007e5', 'B19101e16', 'B11016e12', 'B17020e3', 'B19101e3', 'B25012e14', 'B09002e18', 'B17012e2', 'B19101e2', 
         'B25012e5', 'B17012e5', 'B10010e2', 'B11005e4', 'B19101e15', 'B11005e5', 'B19101e17', 'B17012e4', 'B17012e7', 
         'B20004e8', 'B17020e10', 'B09018e8', 'B20004e6', 'B17020e6', 'B10002e5', 'B11016e11', 'B10002e3', 'B17012e26', 
         'B25012e3', 'B11005e10', 'B08301e5', 'B17012e25', 'B17012e20', 'B08301e4', 'B19101e13', 'B10002e4', 'B23008e13', 
         'C18108e3', 'B09018e7', 'B11001e9', 'B10002e1', 'C18108e4', 'B23008e19', 'B10010e3', 'B16007e7', 'B23008e12', 
         'B15003e23', 'B15003e25', 'B11005e6', 'B11003e7', 'B27003e1', 'B15003e20', 'B09018e1', 'B11016e3', 'B01001e4', 
         'B11016e2', 'B15003e21', 'B15003e22', 'B19083e1', 'B15003e24', 'B23020e1', 'B11016e4', 'B09018e5', 'B19125e2', 
         'B16007e3', 'B01001e28', 'B23020e3', 'B23020e2', 'B12001e6', 'B27001e30', 'B01001e29', 'B25103e2', 'B16007e4', 
         'B12001e9', 'B23008e10', 'B09018e4', 'B01001e30', 'B11003e5', 'B23008e2', 'B23008e5', 'B09002e16', 'B25012e9', 
         'B01002e3', 'B20004e5', 'B25012e2', 'B08303e1', 'B23025e4', 'B23008e4', 'B23025e7', 'B12001e1', 'B15003e17', 
         'B11005e7', 'B01001e3', 'B23025e2', 'B09002e8', 'B22003e5', 'B08301e1']

columns=list(train.columns)
# remove the columns that have been dropped during the previous iterations of the program, before the connection with the kernel stopped
for name in to_drop:
    columns.remove(name)

#ML_MAE is going to store the MAE every time the mixed effects model runs
ML_MAE = {'number_col':[], 'MAE':[]}

while i < (train.shape[1] - len(to_drop)):
    #col contains the features we want to train our model on
    col = columns[:i+1]
    col = '+'.join(col)
    print(i)
    #run the mixed effect model
    try:
        md = smf.mixedlm("unacast_session_count ~ " + col, train, groups=train["climate"])
        mdf = md.fit(method=['bfgs', 'lbfgs', 'cg', 'bfgs'])
    #if the model throws an error when it runs
    except np.linalg.LinAlgError:
        #drop the last feature that was added
        to_drop.append(columns[i])
        print('removed', to_drop[1:])
        columns.remove(columns[i])
    else:
        MAE = metrics.mean_absolute_error(y_train, mdf.predict(train))
        ML_MAE['number_col'].append(i)
        ML_MAE['MAE'].append(MAE)
        i +=1

142


KeyboardInterrupt: 

In [18]:
#Delete the columns found in the previous cell from the list of features
to_delete = ['monthly_repeated_sessions', 'B23008e21', 'B23008e23', 'B12001e11', 'B23008e26', 'B11005e14', 'B11005e17', 
             'B20004e9', 'B19101e7', 'B19101e4', 'B12001e15', 'B09002e19', 'B23008e27', 'B20004e7', 'B11016e10', 'B17012e31', 
             'B19101e8', 'C18108e5', 'B25012e8', 'B25012e15', 'B17020e4', 'B25012e16', 'B25012e10', 'B25012e6', 'B09002e17',
             'B25012e12', 'B25012e11', 'B08301e6', 'B17012e6', 'B19101e6', 'B25012e4', 'B19101e12', 'B17020e5', 'B23008e18', 
             'B19101e5', 'B12001e16', 'B08303e12', 'B17020e11', 'B10010e1', 'B19101e11', 'B08303e13', 'B16007e5', 'B19101e16', 
             'B11016e12', 'B17020e3', 'B19101e3', 'B25012e14', 'B09002e18', 'B17012e2', 'B19101e2', 'B25012e5', 'B17012e5', 
             'B10010e2', 'B11005e4', 'B19101e15', 'B11005e5', 'B19101e17', 'B17012e4', 'B17012e7', 'B20004e8', 'B17020e10', 
             'B09018e8', 'B20004e6', 'B17020e6', 'B10002e5', 'B11016e11', 'B10002e3', 'B17012e26', 'B25012e3', 'B11005e10', 
             'B08301e5', 'B17012e25', 'B17012e20', 'B08301e4', 'B19101e13', 'B10002e4', 'B23008e13', 'C18108e3', 'B09018e7', 
             'B11001e9', 'B10002e1', 'C18108e4', 'B23008e19', 'B10010e3', 'B16007e7', 'B23008e12', 'B15003e23', 'B15003e25', 
             'B11005e6', 'B11003e7', 'B27003e1', 'B15003e20', 'B09018e1', 'B11016e3', 'B01001e4', 'B11016e2', 'B15003e21', 
             'B15003e22', 'B19083e1', 'B15003e24', 'B23020e1', 'B11016e4', 'B09018e5', 'B19125e2', 'B16007e3', 'B01001e28', 
             'B23020e3', 'B23020e2', 'B12001e6', 'B27001e30', 'B01001e29', 'B25103e2', 'B16007e4', 'B12001e9', 'B23008e10', 
             'B09018e4', 'B01001e30', 'B11003e5', 'B23008e2', 'B23008e5', 'B09002e16', 'B25012e9', 'B01002e3', 'B20004e5', 
             'B25012e2', 'B08303e1', 'B23025e4', 'B23008e4', 'B23025e7', 'B12001e1', 'B15003e17', 'B11005e7', 'B01001e3', 
             'B23025e2', 'B09002e8', 'B22003e5', 'B08301e1', 'B14002e1', 'B09002e11', 'B27001e2', 'B25064e1', 'B25001e1', 
             'B19058e2', 'B09001e5', 'B09002e10', 'B09002e13', 'B09002e12', 'B19301e1', 'B25119e2', 'B19125e3', 'B13016e4', 
             'B11001e5', 'B25119e3', 'B15003e18', 'B01001e27', 'B13016e5', 'B11005e9', 'B20004e2', 'B15003e1', 'B10001e4', 
             'B11001e6', 'B13016e7', 'B12001e7', 'B23008e9', 'B13016e6', 'B23008e7', 'B23008e6', 'B09002e14', 'B09001e10', 
             'B11005e8', 'B13016e3', 'male_60_69', 'male_80_over', 'female_22_29', 'female_30_39', 'female_40_49', 
             'female_50_59', 'female_60_69', 'female_70_79', 'female_80_over', 'total_enrolled', 'not_enrolled', 
             'enrolled_nursery_pre_private', 'enrolled_kinder_public', 'enrolled_kinder_private', 'enrolled_grades_1_4_public', 
             'enrolled_grades_1_4_private', 'enrolled_grades_5_8_public', 'enrolled_grades_9_12_private', 
             'enrolled_undergrad_public', 'enrolled_undergrad_private', 'enrolled_graduate_public', 'less_than_12_no_diploma', 
             'travel_15_minutes_less', 'travel_15_29_minutes', 'travel_30_44_minutes', 'single_under_6', 'single_0_to_17', 
             'single_6_to_17', 'single_no_kids', 'four_or_more_in_family_household', 'four_or_more_in_nonfamily_household', 
             'men_without_health_insurance', 'women_with_health_insurnace', 'women_without_health_insurance', 
             'avg_birth_weight', 'avg_age_of_mother', 'longitude', 'latitude', 'alcohol', 'amenity', 'bank', 'bar', 'cafe', 
             'camp_site', 'car_repair', 'childcare', 'clothes_store', 'convenience_store', 'fast_food', 'fire_station', 
             'fitness_or_sports_centre', 'fuel', 'healthcare', 'hotel', 'museum_or_gallery', 'restaurant', 'shop', 
             'supermarket', 'tourism', 'distance_to_U', 'distance_to_nearest_school', 'walk_score', 'bike_score', 'k_avg', 
             'streets_per_node_avg', 'edge_length_total', 'edge_length_avg', 'street_length_avg', 
             'streets_per_node_counts_2', 'streets_per_node_counts_3', 'streets_per_node_counts_5', 'n_osdw', 
             'k_avg_osdw', 'streets_per_node_avg_osdw', 'edge_length_total_osdw', 'streets_per_node_counts_1_osdw', 
             'streets_per_node_counts_3_osdw', 'streets_per_node_counts_5_osdw', 'intersection_count_osid', 
             'edge_length_total_osid', 'edge_length_avg_osid', 'street_length_total_osid', 'street_segments_count_osid', 
             'streets_per_node_counts_1_osid', 'streets_per_node_counts_2_osid', 'streets_per_node_counts_4_osid', 
             'streets_per_node_counts_5_osid', 'violent_crime', 'criminal_homicide', 'rape', 'robbery', 
             'aggravated_assault', 'property_crime', 'burglary', 'larceny_theft', 'motor_vehicle_theft', 
             'houses_per_sq_km', 'historic_number_of_sessions', 'historic_sessions_per_day', 'historic_unique_sessions', 
             'historic_unique_sessions_per_day', 'historic_repeat_sessions', 'historic_repeat_sessions_per_day', 
             'historic_total_session_length', 'historic_avg_session_length', 'historic_avg_light_activity', 
             'historic_avg_moderate_activity', 'historic_avg_vigorous_activity', 'historic_avg_mod_plus_vig', 
             'historic_hour_7', 'historic_hour_8', 'historic_hour_9', 'historic_hour_10', 'historic_hour_11', 
             'historic_hour_13', 'historic_hour_14', 'historic_hour_15', 'historic_hour_16', 'historic_hour_17', 
             'historic_hour_19', 'historic_hour_20', 'historic_rain', 'historic_foggy', 'historic_snow', 'Green_2016', 
             'Libertarians_2016', 'Poor_physical_health_days', 'Adult_smoking', 'Adult_obesity', 'weather_clear', 
             'weather_rain', 'weather_fog', 'temp_avg_35_below', 'temp_max_35_below', 'temp_max_45_55', 
             'state_amount_per_capita', 'historic_slide_count_comb', 'monthly_climb_count_comb', 'monthly_tube_count_comb', 
             'historic_tube_count_comb', 'monthly_overhang_count_comb', 'historic_overhang_count_comb', 
             'monthly_bridge_count_comb', 'historic_bridge_count_comb', 'monthly_swing_count_comb', 
             'historic_swing_count_comb', 'historic_obsta_count_comb', 'historic_crawls_count_comb', 
             'monthly_hour_night', 'historic_hour_night', 'avg_wind_calm', 'avg_wind_light_air', 'avg_wind_light_br', 
             'avg_wind_gentle_br', 'avg_wind_moderate_br', 'monthly_ws_calm', 'monthly_ws_light_air', 'monthly_ws_light_br', 
             'monthly_ws_gentle_br', 'monthly_ws_moderate_br', 'historic_ws_calm', 'historic_ws_light_air', 
             'historic_ws_light_br', 'historic_ws_gentle_br', 'historic_ws_moderate_br', 'avg_fertility_rate', 'HI', 'LI', 
             'MI', 'HD', 'LD', 'MD']
train = train.drop(columns = to_delete)
valid = X_valid.drop(columns=to_delete)

In [19]:
len(list(train.columns))

286

In [21]:
#The new set of columns we are going to work with
cols =  '+'.join(list(train.columns)[:-2])

In [22]:
list(train.columns)[-2:]

['climate', 'unacast_session_count']

In [80]:
train_var.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39592 entries, 28601 to 42347
Columns: 286 entries, month to climate
dtypes: float64(285), object(1)
memory usage: 87.9+ MB


In [34]:
# In this cell, all the features that are deleted were found in the cell above that tried to find all the columns that make the model not converge
def run_MLE(group=[]):
    """
    Run the Mixed effects model using the grouped passed as an argument
    
    Parameters
    ---------     
    group: list containing one categorical column to use to cluster the playgrounds
    
    Returns
    -------
    tuple with 2 lists : the first list contains the train and validation RMSE, the seconde one contains the train and validation MAE
    """
    #preprocess the data
    X_train_valid_y=preprocessing(group)
    X_train_valid =  X_train_valid_y[0]
    y_train =  X_train_valid_y[1]
    y_valid =  X_train_valid_y[2]
    X_train = X_train_valid[0]
    X_valid = X_train_valid[1]
    train = pd.concat([X_train, y_train], axis=1)
    to_delete = ['monthly_repeated_sessions', 'B23008e21', 'B23008e23', 'B12001e11', 'B23008e26', 'B11005e14', 'B11005e17', 
                 'B20004e9', 'B19101e7', 'B19101e4', 'B12001e15', 'B09002e19', 'B23008e27', 'B20004e7', 'B11016e10', 'B17012e31', 
                 'B19101e8', 'C18108e5', 'B25012e8', 'B25012e15', 'B17020e4', 'B25012e16', 'B25012e10', 'B25012e6', 'B09002e17',
                 'B25012e12', 'B25012e11', 'B08301e6', 'B17012e6', 'B19101e6', 'B25012e4', 'B19101e12', 'B17020e5', 'B23008e18', 
                 'B19101e5', 'B12001e16', 'B08303e12', 'B17020e11', 'B10010e1', 'B19101e11', 'B08303e13', 'B16007e5', 'B19101e16', 
                 'B11016e12', 'B17020e3', 'B19101e3', 'B25012e14', 'B09002e18', 'B17012e2', 'B19101e2', 'B25012e5', 'B17012e5', 
                 'B10010e2', 'B11005e4', 'B19101e15', 'B11005e5', 'B19101e17', 'B17012e4', 'B17012e7', 'B20004e8', 'B17020e10', 
                 'B09018e8', 'B20004e6', 'B17020e6', 'B10002e5', 'B11016e11', 'B10002e3', 'B17012e26', 'B25012e3', 'B11005e10', 
                 'B08301e5', 'B17012e25', 'B17012e20', 'B08301e4', 'B19101e13', 'B10002e4', 'B23008e13', 'C18108e3', 'B09018e7', 
                 'B11001e9', 'B10002e1', 'C18108e4', 'B23008e19', 'B10010e3', 'B16007e7', 'B23008e12', 'B15003e23', 'B15003e25', 
                 'B11005e6', 'B11003e7', 'B27003e1', 'B15003e20', 'B09018e1', 'B11016e3', 'B01001e4', 'B11016e2', 'B15003e21', 
                 'B15003e22', 'B19083e1', 'B15003e24', 'B23020e1', 'B11016e4', 'B09018e5', 'B19125e2', 'B16007e3', 'B01001e28', 
                 'B23020e3', 'B23020e2', 'B12001e6', 'B27001e30', 'B01001e29', 'B25103e2', 'B16007e4', 'B12001e9', 'B23008e10', 
                 'B09018e4', 'B01001e30', 'B11003e5', 'B23008e2', 'B23008e5', 'B09002e16', 'B25012e9', 'B01002e3', 'B20004e5', 
                 'B25012e2', 'B08303e1', 'B23025e4', 'B23008e4', 'B23025e7', 'B12001e1', 'B15003e17', 'B11005e7', 'B01001e3', 
                 'B23025e2', 'B09002e8', 'B22003e5', 'B08301e1', 'B14002e1', 'B09002e11', 'B27001e2', 'B25064e1', 'B25001e1', 
                 'B19058e2', 'B09001e5', 'B09002e10', 'B09002e13', 'B09002e12', 'B19301e1', 'B25119e2', 'B19125e3', 'B13016e4', 
                 'B11001e5', 'B25119e3', 'B15003e18', 'B01001e27', 'B13016e5', 'B11005e9', 'B20004e2', 'B15003e1', 'B10001e4', 
                 'B11001e6', 'B13016e7', 'B12001e7', 'B23008e9', 'B13016e6', 'B23008e7', 'B23008e6', 'B09002e14', 'B09001e10', 
                 'B11005e8', 'B13016e3', 'male_60_69', 'male_80_over', 'female_22_29', 'female_30_39', 'female_40_49', 
                 'female_50_59', 'female_60_69', 'female_70_79', 'female_80_over', 'total_enrolled', 'not_enrolled', 
                 'enrolled_nursery_pre_private', 'enrolled_kinder_public', 'enrolled_kinder_private', 'enrolled_grades_1_4_public', 
                 'enrolled_grades_1_4_private', 'enrolled_grades_5_8_public', 'enrolled_grades_9_12_private', 
                 'enrolled_undergrad_public', 'enrolled_undergrad_private', 'enrolled_graduate_public', 'less_than_12_no_diploma', 
                 'travel_15_minutes_less', 'travel_15_29_minutes', 'travel_30_44_minutes', 'single_under_6', 'single_0_to_17', 
                 'single_6_to_17', 'single_no_kids', 'four_or_more_in_family_household', 'four_or_more_in_nonfamily_household', 
                 'men_without_health_insurance', 'women_with_health_insurnace', 'women_without_health_insurance', 
                 'avg_birth_weight', 'avg_age_of_mother', 'longitude', 'latitude', 'alcohol', 'amenity', 'bank', 'bar', 'cafe', 
                 'camp_site', 'car_repair', 'childcare', 'clothes_store', 'convenience_store', 'fast_food', 'fire_station', 
                 'fitness_or_sports_centre', 'fuel', 'healthcare', 'hotel', 'museum_or_gallery', 'restaurant', 'shop', 
                 'supermarket', 'tourism', 'distance_to_U', 'distance_to_nearest_school', 'walk_score', 'bike_score', 'k_avg', 
                 'streets_per_node_avg', 'edge_length_total', 'edge_length_avg', 'street_length_avg', 
                 'streets_per_node_counts_2', 'streets_per_node_counts_3', 'streets_per_node_counts_5', 'n_osdw', 
                 'k_avg_osdw', 'streets_per_node_avg_osdw', 'edge_length_total_osdw', 'streets_per_node_counts_1_osdw', 
                 'streets_per_node_counts_3_osdw', 'streets_per_node_counts_5_osdw', 'intersection_count_osid', 
                 'edge_length_total_osid', 'edge_length_avg_osid', 'street_length_total_osid', 'street_segments_count_osid', 
                 'streets_per_node_counts_1_osid', 'streets_per_node_counts_2_osid', 'streets_per_node_counts_4_osid', 
                 'streets_per_node_counts_5_osid', 'violent_crime', 'criminal_homicide', 'rape', 'robbery', 
                 'aggravated_assault', 'property_crime', 'burglary', 'larceny_theft', 'motor_vehicle_theft', 
                 'houses_per_sq_km', 'historic_number_of_sessions', 'historic_sessions_per_day', 'historic_unique_sessions', 
                 'historic_unique_sessions_per_day', 'historic_repeat_sessions', 'historic_repeat_sessions_per_day', 
                 'historic_total_session_length', 'historic_avg_session_length', 'historic_avg_light_activity', 
                 'historic_avg_moderate_activity', 'historic_avg_vigorous_activity', 'historic_avg_mod_plus_vig', 
                 'historic_hour_7', 'historic_hour_8', 'historic_hour_9', 'historic_hour_10', 'historic_hour_11', 
                 'historic_hour_13', 'historic_hour_14', 'historic_hour_15', 'historic_hour_16', 'historic_hour_17', 
                 'historic_hour_19', 'historic_hour_20', 'historic_rain', 'historic_foggy', 'historic_snow', 'Green_2016', 
                 'Libertarians_2016', 'Poor_physical_health_days', 'Adult_smoking', 'Adult_obesity', 'weather_clear', 
                 'weather_rain', 'weather_fog', 'temp_avg_35_below', 'temp_max_35_below', 'temp_max_45_55', 
                 'state_amount_per_capita', 'historic_slide_count_comb', 'monthly_climb_count_comb', 'monthly_tube_count_comb', 
                 'historic_tube_count_comb', 'monthly_overhang_count_comb', 'historic_overhang_count_comb', 
                 'monthly_bridge_count_comb', 'historic_bridge_count_comb', 'monthly_swing_count_comb', 
                 'historic_swing_count_comb', 'historic_obsta_count_comb', 'historic_crawls_count_comb', 
                 'monthly_hour_night', 'historic_hour_night', 'avg_wind_calm', 'avg_wind_light_air', 'avg_wind_light_br', 
                 'avg_wind_gentle_br', 'avg_wind_moderate_br', 'monthly_ws_calm', 'monthly_ws_light_air', 'monthly_ws_light_br', 
                 'monthly_ws_gentle_br', 'monthly_ws_moderate_br', 'historic_ws_calm', 'historic_ws_light_air', 
                 'historic_ws_light_br', 'historic_ws_gentle_br', 'historic_ws_moderate_br', 'avg_fertility_rate', 'HI', 'LI', 
                 'MI', 'HD', 'LD', 'MD', 'A', 'C', 'D']
    
    if 'income_class' in group:
        for feat in ['MI', 'HD', 'LD', 'MD', 'HI', 'LI']:
            to_delete.remove(feat)
    elif 'density_class' in group:
        for feat in ['HD', 'LD', 'MD']:
            to_delete.remove(feat)
    elif 'climate' in group:
        for feat in ['A', 'C', 'D']:
            to_delete.remove(feat)
    
    train = train.drop(columns = to_delete)
    valid = X_valid.drop(columns=to_delete)
    
    cols = list(train.columns)
    cols.remove('unacast_session_count')
    for col in group:
        cols.remove(col)
    cols =  '+'.join(cols)
    #run the model
    md = smf.mixedlm('unacast_session_count ~ ' + cols, data = train, groups=train[group[0]])
    mdf = md.fit(method=['bfgs', 'lbfgs', 'cg', 'bfgs'])
    
    return(report_performance(mdf, X_train, y_train, X_valid, y_valid, mode='mean', floor=True),
    report_performance(mdf, X_train, y_train, X_valid, y_valid, mode='median', floor=True))

In [23]:
run_MLE(['climate'])

['climate', 'unacast_session_count']


/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
/ho

Training root mean squared error: 198.08289093303287
Validation root mean squared error: 203.1055499417757
Training mean absolute error: 99.0338783002849
Validation mean absolute error: 99.73981748123848


([198.08289093303287, 203.1055499417757],
 [99.0338783002849, 99.73981748123848])

In [35]:
results={'group':[], 'data':[], 'type':[], 'error':[]}
list_type=['RMSE', 'MAE']
list_data=['train', 'validation']
for var in [['income_class'], ['density_class'], ['climate']]:
    error = run_MLE(var)
    for i in range(2):
        for j in range(2):
            results['group'].append(var[0])
            results['data'].append(list_data[j])
            results['type'].append(list_type[i])
            results['error'].append(error[i][j])

/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
/ho

Training root mean squared error: 198.49880816751818
Validation root mean squared error: 203.87057935663918
Training mean absolute error: 101.07085398250304
Validation mean absolute error: 101.8901470520564


/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
/ho

Training root mean squared error: 198.52776081779135
Validation root mean squared error: 203.55110773018416
Training mean absolute error: 100.06061172546366
Validation mean absolute error: 100.68693529498546


/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
/ho

Training root mean squared error: 198.08289093303287
Validation root mean squared error: 203.1055499417757
Training mean absolute error: 99.0338783002849
Validation mean absolute error: 99.73981748123848


In [36]:
pd.DataFrame(results)

,group,data,type,error
0,income_class,train,RMSE,198.498808
1,income_class,validation,RMSE,203.870579
2,income_class,train,MAE,101.070854
3,income_class,validation,MAE,101.890147
4,density_class,train,RMSE,198.527761
5,density_class,validation,RMSE,203.551108
6,density_class,train,MAE,100.060612
7,density_class,validation,MAE,100.686935
8,climate,train,RMSE,198.082891
9,climate,validation,RMSE,203.105550


**Conclusion** : So far, I only ran the MLE model grouping by the 'income_class', 'density_class' and 'climate' features, but the results were not very good (validation RMSE around 200 and validation MAE around 100). Future improvements could be made, for example it could be a good idea to try to cluster the playgrounds using more than one feature (or use some numerical features and binning them)